In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score

from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans

In [4]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')
df19 = pd.read_csv('../data/semana_3_dias_antes_features.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
# Al que tenga "diferencia_de_count_de_eventos.csv" hay que rellenarlo
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17, how='inner')
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df19, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 102)

In [5]:
features = df_test.columns.tolist()
features.remove('label')

In [6]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [7]:
param_grid = { 
    'n_estimators': [x for x in range(100, 1000, 100)],
    'gamma' : [1,2,3,4,5],
}

In [8]:
rf_param_grid = {
    'n_estimators': [x for x in range(1000, 3000, 500)],
    'max_depth': [3,5,7],
    'min_samples_split': [100,200,300]
}

In [9]:
rfc = RandomForestClassifier(class_weight='balanced', random_state=80)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=rf_param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10)
CV_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 26.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 45.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 54.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 63.3min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 72.9min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 78.3min finished

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=80,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [1000, 1500, 2000, 2500], 'max_depth': [3, 5, 7], 'min_samples_split': [100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [10]:
CV_rfc.best_params_

{'max_depth': 7, 'min_samples_split': 300, 'n_estimators': 2500}

In [11]:
CV_rfc.best_score_

0.8686724575620235

In [12]:
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances = pd.DataFrame(CV_rfc.best_estimator_.feature_importances_,\
                                   index = X.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(40)

,importance
storage_mas_checkout,0.120287
tasa_ult_quin_cant_checkouts,0.118731
ult_quin_cant_checkouts,0.109777
cant_checkouts,0.059362
tiene_checkouts,0.051270
model_encoding,0.048941
ult_checkout,0.042395
cant_viewed_product,0.031098
suma_total,0.030407
days_elapsed,0.030153


In [13]:
top_40_features = feature_importances.index.tolist()[:40]

### Reducción con K-Means

Voy a reducir las dimensiones de las 40 features usando kmeans y veremos que pasa...

In [126]:
k_means = KMeans(n_clusters=30, random_state=420)

In [127]:
top_40_features_reduced = k_means.fit_transform(df_test[top_40_features])

In [128]:
top_40_features_reduced = pd.DataFrame(top_40_features_reduced)

In [129]:
top_40_features_reduced['person'] = df_test.reset_index()['person']

In [130]:
top_40_features_reduced.set_index('person', inplace=True)

In [131]:
df_test_new =  (top_40_features_reduced - top_40_features_reduced.mean()) / top_40_features_reduced.std()

In [132]:
df_test_new['label'] = df_test['label']

Queda comentada la parte que reseteo el set de test al top 40 directo sin el K-Means.

In [110]:
# df_test_new = df_test[top_40_features]
# df_test_new = (df_test_new - df_test_new.mean()) / df_test_new.std()
# df_test_new['person'] = df_test.reset_index()['person']
# df_test_new['label'] = df_test['label']

In [119]:
# df_test_new.fillna(0, inplace=True)

In [133]:
features = df_test_new.columns.tolist()
features.remove('label')

X = df_test_new[features]
Y = df_test_new['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [134]:
params = {
    'hidden_layer_sizes': [(20, 11, 1), (21, 10, 1), (25, 50, 25)],
    'alpha': [0.0001, 0.001],
    'activation': ['logistic', 'relu']
}

In [135]:
mlp = MLPRegressor(random_state=420)
CV_mlp = GridSearchCV(estimator=mlp, param_grid=params, cv=5, scoring='roc_auc', n_jobs=-1, verbose=10)
CV_mlp.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=420,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(20, 11, 1), (21, 10, 1), (25, 50, 25)], 'alpha': [0.0001, 0.001], 'activation': ['logistic', 'relu']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [136]:
CV_mlp.best_params_

{'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (25, 50, 25)}

In [137]:
CV_mlp.best_score_

0.81321157961314

In [138]:
print(roc_auc_score(y_test, CV_mlp.predict(X_test)))

0.7925104657247515


In [139]:
df_submit.to_csv('../submit_mlp_kmeans_reduccion.csv')

NameError: name 'df_submit' is not defined